# Tiktok Video Enhancer

Tiktok Video Enhancer es una solución completa para mejorar la calidad de audio y video en archivos multimedia. Este script de Python extrae el audio del video, mejora su calidad eliminando ruido, ajusta el brillo y contraste del video, y luego reintegra el audio mejorado con el video original para producir un resultado de alta calidad.

#### Instalación de librerías

In [1]:
pip install -q pydub ffmpeg noise numpy scipy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
pip install -q noisereduce

#### v1 Mejora calidad audio

In [7]:
from pydub import AudioSegment
from pydub.effects import normalize, low_pass_filter, high_pass_filter
import numpy as np
import noisereduce as nr
import wave
import os

def reduce_noise(input_audio_path, output_audio_path):
    # Load audio
    audio = AudioSegment.from_file(input_audio_path)
    audio = audio.set_channels(1)  # Convert to mono for processing
    audio.export("temp.wav", format="wav")

    # Open wave file
    with wave.open("temp.wav", 'r') as wav_file:
        n_frames = wav_file.getnframes()
        framerate = wav_file.getframerate()
        audio_data = wav_file.readframes(n_frames)
        audio_np = np.frombuffer(audio_data, dtype=np.int16)

    # Reduce noise
    reduced_noise = nr.reduce_noise(y=audio_np, sr=framerate)

    # Save output
    reduced_audio = AudioSegment(
        np.array(reduced_noise, dtype=np.int16).tobytes(),
        frame_rate=framerate,
        sample_width=2,  # 16-bit audio
        channels=1
    )
    reduced_audio.export(output_audio_path, format="wav")

def enhance_audio(input_audio_path, output_audio_path):
    # Load audio
    audio = AudioSegment.from_file(input_audio_path)

    # Apply a high-pass filter to remove low-frequency noise (e.g., hum)
    audio = high_pass_filter(audio, cutoff=100)  # Remove frequencies below 100Hz

    # Apply a low-pass filter to remove high-frequency noise
    audio = low_pass_filter(audio, cutoff=8000)  # Remove frequencies above 8000Hz

    # Normalize audio to even out the volume levels
    audio = normalize(audio)

    # Optionally increase the bit depth for higher quality (e.g., 24-bit)
    audio = audio.set_frame_rate(48000)  # Upsample to 48kHz if needed

    # Export the enhanced audio
    audio.export(output_audio_path, format="wav", bitrate="320k")
    print(f"Enhanced audio saved to {output_audio_path}")

def process_audio_and_merge_video(input_video_path, output_video_path):
    # Extract audio from the video
    extracted_audio_path = "extracted_audio.wav"
    os.system(f"ffmpeg -i {input_video_path} -q:a 0 -map a {extracted_audio_path} -y")

    # Reduce noise
    noise_reduced_audio_path = "noise_reduced.wav"
    reduce_noise(extracted_audio_path, noise_reduced_audio_path)

    # Enhance audio quality
    enhanced_audio_path = "enhanced_audio.wav"
    enhance_audio(noise_reduced_audio_path, enhanced_audio_path)

    # Merge enhanced audio back into the video
    os.system(f"ffmpeg -i {input_video_path} -i {enhanced_audio_path} -c:v copy -map 0:v:0 -map 1:a:0 {output_video_path} -y")
    print(f"Processed video saved to {output_video_path}")

# Example usage:
input_video_path = "/content/1ade941accb54bce852af7d6c83940a2.mov"  # Replace with your video file path
output_video_path = "/content/processed_video_high_quality.mp4"

process_audio_and_merge_video(input_video_path, output_video_path)


Enhanced audio saved to enhanced_audio.wav
Enhanced video saved to enhanced_video.mp4
Final processed video saved to /content/processed_video_high_quality.mp4


### v2: Mejora calidad video y audio

In [ ]:
from pydub import AudioSegment
from pydub.effects import normalize, low_pass_filter, high_pass_filter
import numpy as np
import noisereduce as nr
import wave
import os


def reduce_noise(input_audio_path, output_audio_path):
    """
    Reduces noise in an audio file and saves the result.

    Parameters:
    - input_audio_path (str): Path to the input audio file.
    - output_audio_path (str): Path to save the noise-reduced audio file.
    """
    # Load audio and convert to mono
    audio = AudioSegment.from_file(input_audio_path)
    audio = audio.set_channels(1)
    audio.export("temp.wav", format="wav")

    # Read the temporary WAV file
    with wave.open("temp.wav", 'r') as wav_file:
        n_frames = wav_file.getnframes()
        framerate = wav_file.getframerate()
        audio_data = wav_file.readframes(n_frames)
        audio_np = np.frombuffer(audio_data, dtype=np.int16)

    # Reduce noise using noisereduce library
    reduced_noise = nr.reduce_noise(y=audio_np, sr=framerate)

    # Save the noise-reduced audio
    reduced_audio = AudioSegment(
        np.array(reduced_noise, dtype=np.int16).tobytes(),
        frame_rate=framerate,
        sample_width=2,  # 16-bit audio
        channels=1
    )
    reduced_audio.export(output_audio_path, format="wav")


def enhance_audio(input_audio_path, output_audio_path):
    """
    Enhances the audio quality by applying filters and normalization.

    Parameters:
    - input_audio_path (str): Path to the input audio file.
    - output_audio_path (str): Path to save the enhanced audio file.
    """
    # Load the audio file
    audio = AudioSegment.from_file(input_audio_path)

    # Apply high-pass filter to remove low-frequency noise
    audio = high_pass_filter(audio, cutoff=100)

    # Apply low-pass filter to remove high-frequency noise
    audio = low_pass_filter(audio, cutoff=8000)

    # Normalize the audio to even out volume levels
    audio = normalize(audio)

    # Export the enhanced audio with high bitrate
    audio.export(output_audio_path, format="wav", bitrate="320k")
    print(f"Enhanced audio saved to {output_audio_path}")


def enhance_video(input_video_path, output_video_path):
    """
    Enhances the video quality by applying denoise, scaling, and color adjustments.

    Parameters:
    - input_video_path (str): Path to the input video file.
    - output_video_path (str): Path to save the enhanced video file.
    """
    # Apply filters using FFmpeg
    command = (
        f"ffmpeg -i {input_video_path} "
        f"-vf hqdn3d=3:2:2:3,scale=1920:1080,eq=contrast=1.1:brightness=0.05 "
        f"-c:v libx264 -preset slow -crf 18 {output_video_path} -y"
    )
    os.system(command)
    print(f"Enhanced video saved to {output_video_path}")


def process_audio_and_video(input_video_path, output_video_path):
    """
    Processes the audio of a video file by reducing noise, enhancing quality,
    enhancing video, and merging the enhanced audio back into the video.

    Parameters:
    - input_video_path (str): Path to the input video file.
    - output_video_path (str): Path to save the processed video file.
    """
    # Extract audio from the video
    extracted_audio_path = "extracted_audio.wav"
    os.system(f"ffmpeg -i {input_video_path} -q:a 0 -map a {extracted_audio_path} -y")

    # Reduce noise and enhance audio
    noise_reduced_audio_path = "noise_reduced.wav"
    reduce_noise(extracted_audio_path, noise_reduced_audio_path)

    enhanced_audio_path = "enhanced_audio.wav"
    enhance_audio(noise_reduced_audio_path, enhanced_audio_path)

    # Enhance video
    enhanced_video_path = "enhanced_video.mp4"
    enhance_video(input_video_path, enhanced_video_path)

    # Merge the enhanced audio back into the enhanced video
    os.system(
        f"ffmpeg -i {enhanced_video_path} -i {enhanced_audio_path} -c:v copy "
        f"-map 0:v:0 -map 1:a:0 {output_video_path} -y"
    )
    print(f"Final processed video saved to {output_video_path}")


# Example usage
if __name__ == "__main__":
    input_video_path = "/content/1ade941accb54bce852af7d6c83940a2.mov"
    output_video_path = "/content/processed_video_high_quality.mp4"
    process_audio_and_video(input_video_path, output_video_path)
